# Setup Environment in Google Colab
Configure Google Colab environment, check for GPU availability, and mount Google Drive for storage.

In [ ]:
# Check if GPU is available
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available')
else:
    device = torch.device('cpu')
    print('GPU is not available, using CPU')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory to the project folder in Google Drive
import os
project_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification'
if not os.path.exists(project_path):
    os.makedirs(project_path)
os.chdir(project_path)
print(f'Changed working directory to {project_path}')

# Install YOLOv8
Install the Ultralytics YOLOv8 package and other required dependencies.

In [ ]:
# Install YOLOv8
!pip install ultralytics

# Verify the installation
!yolo checks

# Prepare Clothing Dataset
Download deepFashion 2 dataset


In [ ]:
# Download and prepare the iMaterialist Fashion 2019 dataset for classification and segmentation
import gdown
import zipfile

# Download the dataset from Google Drive
%pip install gdown -q

# Define dataset directory
data_dir = os.path.join(project_path, 'data')

# Create data directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Created directory: {data_dir}")

# Download the dataset from Google Drive
url = "http://drive.google.com/drive/folders/125F48fsMBz2EF0Cpqk6aaHet5VH399Ok"
output = os.path.join(data_dir, 'clothing_dataset.zip')

print("Downloading dataset from Google Drive...")
try:
    # Use gdown to download the folder
    gdown.download_folder(url, output=data_dir, quiet=False)
    print("Download completed successfully!")
except Exception as e:
    print(f"Error downloading or extracting dataset: {e}")

# Verify the download
print("\nContents of data directory:")
!ls -la {data_dir}

Turn deepfasion2 data to yolo format
convert_deepfashion2_to_yolo(input_dir,images_dir, output_dir)

In [ ]:
#for testing purpose
!python deepfashion2_to_yolo.py datasets/coco/train_subset/annos datasets/coco/train_subset/images datasets/yolo/train/labels
!python deepfashion2_to_yolo.py datasets/coco/val_subset/annos datasets/coco/val_subset/images datasets/yolo/val/labels

Conversion completed. YOLO format annotations saved to datasets/yolo/train/labels
Conversion completed. YOLO format annotations saved to datasets/yolo/val/labels


In [13]:
%%bash

# Define source and destination directories
source_dir="train/train/image"
destination_dir="datasets/train_subset/images"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.jpg"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 images from $source_dir to $destination_dir."

Copied the first 10,000 images from train/train/image to datasets/train_subset/images.


In [14]:
%%bash

# Define source and destination directories
source_dir="train/train/annos"
destination_dir="datasets/train_subset/annos"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.json"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 annos from $source_dir to $destination_dir."

Copied the first 10,000 annos from train/train/annos to datasets/train_subset/annos.


In [15]:
%%bash

# Define source and destination directories
source_dir="validation/validation/image"
destination_dir="datasets/val_subset/images"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.jpg"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 images from $source_dir to $destination_dir."

Copied the first 10,000 images from validation/validation/image to datasets/val_subset/images.


In [16]:
%%bash

# Define source and destination directories
source_dir="validation/validation/annos"
destination_dir="datasets/val_subset/annos"

# Create the destination directory if it doesn't exist
mkdir -p "$destination_dir"

# Find the first 10,000 images and copy them to the destination directory
find "$source_dir" -type f -name "*.json"| sort | head -n 10000 | xargs -I {} cp {} "$destination_dir"

echo "Copied the first 10,000 annos from $source_dir to $destination_dir."

Copied the first 10,000 annos from validation/validation/annos to datasets/val_subset/annos.


In [19]:
#convert the subset to yolo format
!python deepfashion2_to_yolo.py datasets/train_subset/annos datasets/train_subset/images datasets/train_subset/labels
!python deepfashion2_to_yolo.py datasets/val_subset/annos datasets/val_subset/images datasets/val_subset/labels

Conversion completed. YOLO format annotations saved to datasets/train_subset/labels
Conversion completed. YOLO format annotations saved to datasets/val_subset/labels


# Train the Segmentation Model
Train YOLOv8 segmentation model to identify clothing items and create masks for background removal.

In [20]:
# Train the Segmentation Model

# Import necessary libraries
import torch
from ultralytics import YOLO

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available')
else:
    device = torch.device('cpu')
    print('GPU is not available, using CPU')


# Initialize the YOLOv8 model for segmentation
model_segmentation = YOLO('yolov8n-seg.pt')  # Use a pre-trained YOLOv8 model for segmentation

# Configure the model for training
model_segmentation.train(data='yoloCOCO.yaml', epochs=50, imgsz=640)

print('YOLOv8 segmentation model training completed.')

GPU is not available, using CPU
New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.11.1 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13600K)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=yoloCOCO.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, em

train: Scanning C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\labels... 10000 images, 0 backgrounds, 56 corrupt: 100%|██████████| 10000/10000 [00:03<00:00, 3096.55it/s]

train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000013.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000014.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000114.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000115.jpg: 2 duplicate labels removed
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000117.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000122.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\000129.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.001069]
train: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\images\00022

train: New cache created: C:\Users\Ben Wu\Desktop\Yolo Training\datasets\train_subset\labels.cache


val: Scanning C:\Users\Ben Wu\Desktop\Yolo Training\datasets\val_subset\labels... 10000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10000/10000 [00:03<00:00, 3099.09it/s]

val: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\val_subset\images\000643.jpg: 1 duplicate labels removed
val: WARNING  C:\Users\Ben Wu\Desktop\Yolo Training\datasets\val_subset\images\007413.jpg: 1 duplicate labels removed


val: New cache created: C:\Users\Ben Wu\Desktop\Yolo Training\datasets\val_subset\labels.cache
Plotting labels to runs\segment\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.472      4.716      4.095      1.607        127        640:  17%|█▋        | 106/622 [04:20<21:09,  2.46s/it]


KeyboardInterrupt: 

# Evaluate Model Performance
Evaluate both models using metrics like precision, recall, mAP, and visualization tools.

In [ ]:
# Evaluate Model Performance

# Import necessary libraries
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Function to evaluate classification model
def evaluate_classification_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.names)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

# Function to evaluate segmentation model
def evaluate_segmentation_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.names)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

# Assuming data loaders for validation data are available as val_loader_classification and val_loader_segmentation
# Evaluate the classification model
evaluate_classification_model(model_classification, val_loader_classification)

# Evaluate the segmentation model
evaluate_segmentation_model(model_segmentation, val_loader_segmentation)

# Perform Background Removal
Implement a pipeline to use the trained segmentation model to remove backgrounds from clothing images.

In [ ]:
# Perform Background Removal

# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Function to remove background using the segmentation model
def remove_background(image_path, model):
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Preprocess the image for the model
    input_image = cv2.resize(image_rgb, (640, 640))
    input_image = input_image / 255.0
    input_image = np.transpose(input_image, (2, 0, 1))
    input_image = np.expand_dims(input_image, axis=0)
    input_image = torch.tensor(input_image, dtype=torch.float32).to(device)

    # Get the segmentation mask from the model
    with torch.no_grad():
        output = model(input_image)[0]
    mask = torch.argmax(output, dim=1).cpu().numpy()[0]

    # Resize the mask to the original image size
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Create a binary mask
    binary_mask = (mask > 0).astype(np.uint8)

    # Apply the mask to the image
    result = cv2.bitwise_and(image_rgb, image_rgb, mask=binary_mask)

    return result

# Example usage
image_path = 'path_to_your_image.jpg'  # Replace with the path to your image
result = remove_background(image_path, model_segmentation)

# Display the original image and the result
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Background Removed')
plt.imshow(result)
plt.axis('off')

plt.show()

# Export and Save the Models
Export the trained models in appropriate formats (ONNX, TorchScript, etc.) and save them to Google Drive for future use.

In [ ]:
# Export and Save the Models

# Export the trained classification model to ONNX format
classification_onnx_path = os.path.join(project_path, 'yolov8_classification.onnx')
model_classification.export(format='onnx', path=classification_onnx_path)
print(f'Classification model exported to {classification_onnx_path}')

# Export the trained segmentation model to ONNX format
segmentation_onnx_path = os.path.join(project_path, 'yolov8_segmentation.onnx')
model_segmentation.export(format='onnx', path=segmentation_onnx_path)
print(f'Segmentation model exported to {segmentation_onnx_path}')

# Save the trained classification model to TorchScript format
classification_torchscript_path = os.path.join(project_path, 'yolov8_classification.pt')
model_classification.save(path=classification_torchscript_path)
print(f'Classification model saved to {classification_torchscript_path}')

# Save the trained segmentation model to TorchScript format
segmentation_torchscript_path = os.path.join(project_path, 'yolov8_segmentation.pt')
model_segmentation.save(path=segmentation_torchscript_path)
print(f'Segmentation model saved to {segmentation_torchscript_path}')

# Save models to Google Drive
drive_classification_onnx_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_classification.onnx'
drive_segmentation_onnx_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_segmentation.onnx'
drive_classification_torchscript_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_classification.pt'
drive_segmentation_torchscript_path = '/content/drive/MyDrive/YOLOv8-Clothing-Classification/yolov8_segmentation.pt'

shutil.copy(classification_onnx_path, drive_classification_onnx_path)
shutil.copy(segmentation_onnx_path, drive_segmentation_onnx_path)
shutil.copy(classification_torchscript_path, drive_classification_torchscript_path)
shutil.copy(segmentation_torchscript_path, drive_segmentation_torchscript_path)

print('Models saved to Google Drive.')